The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [51]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [52]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [53]:
dataset = "w8a"
prob = defineProb(dataset);

../data/w8a.jld


In [54]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [66]:
# Pkg.add("Hadamard")
# using Hadamard 

m = 64;
idx = sample(1:m,10,replace=true);
A = rand(m,m)
mat = hada(A,idx);


H = hadamard(m);
HA =H*A;
error = norm(HA[idx,:] -mat)

1.0081635971830879e-14

In [59]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));

output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
 15  | 2128915606517.06  |  63575495100.66  | 10.3650 


In [29]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
  5  | 95.41  |  1.61  | 0.3709 
 10  | 83.15  |  0.74  | 0.3733 
 15  | 71.34  |  0.56  | 0.3734 
 20  | 57.75  |  0.40  | 0.3736 
 25  | 52.65  |  0.24  | 0.3738 
 30  | 47.07  |  0.20  | 0.3740 
 35  | 43.81  |  0.13  | 0.3742 
 40  | 41.34  |  0.09  | 0.3743 
 45  | 39.50  |  0.08  | 0.3745 
 50  | 36.20  |  0.06  | 0.3747 
 55  | 34.79  |  0.05  | 0.3749 
 60  | 33.18  |  0.04  | 0.3751 
 65  | 31.92  |  0.03  | 0.3753 
 70  | 30.06  |  0.03  | 0.3755 
 75  | 27.95  |  0.03  | 0.3757 
 80  | 25.21  |  0.03  | 0.3759 
 85  | 22.48  |  0.04  | 0.3760 
 90  | 19.45  |  0.03  | 0.3762 
 95  | 17.52  |  0.03  | 0.3763 
100  | 16.05  |  0.02  | 0.3765 
105  | 15.52  |  0.02  | 0.3767 
110  | 14.85  |  0.01  | 0.3769 
115  | 14.03  |  0.02  | 0.3770 
120  | 13.10  |  0.04  | 0.3772 
125  | 12.45  |  0.01  | 0.3774 
130  | 11.38  |  0.03  | 0.3838 
135  | 10.64  |  0.01  | 0.3865 
140  | 9.55  |  0.01  | 0.3886

In [30]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
  5  | 98.18  |  3.14  | 0.0325 
 10  | 97.08  |  4.72  | 0.0374 
 15  | 96.39  |  1.33  | 0.0381 
 20  | 95.56  |  2.82  | 0.0388 
 25  | 94.99  |  0.92  | 0.0398 
 30  | 94.27  |  2.16  | 0.0404 
 35  | 93.76  |  0.75  | 0.0500 
 40  | 93.10  |  1.82  | 0.0643 
 45  | 92.63  |  0.64  | 0.0673 
 50  | 92.02  |  1.60  | 0.0679 
 55  | 91.58  |  0.57  | 0.0685 
 60  | 91.01  |  1.45  | 0.0690 
 65  | 90.59  |  0.52  | 0.0696 
 70  | 90.05  |  1.33  | 0.0701 
 75  | 89.66  |  0.48  | 0.0707 
 80  | 89.14  |  1.23  | 0.0713 
 85  | 88.77  |  0.45  | 0.0718 
 90  | 88.28  |  1.15  | 0.0724 
 95  | 87.92  |  0.42  | 0.0731 
100  | 87.45  |  1.08  | 0.0750 
105  | 87.11  |  0.40  | 0.0755 
110  | 86.65  |  1.03  | 0.0760 
115  | 86.32  |  0.38  | 0.0765 
120  | 85.88  |  0.98  | 0.0770 
125  | 85.57  |  0.36  | 0.0776 
130  | 85.14  |  0.93  | 0.0781 
135  | 84.83  |  0.34  | 0.0785 
140  | 84.42  |  0.89  | 0.0

In [20]:
# Testing CD 
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=600;
method_name = "CD";# gradient descent
options.sketchsize = floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 69.21  |  7.94  | 0.0159 
 50  | 52.38  |  6.15  | 0.0236 
 75  | 47.85  |  3.94  | 0.0339 
100  | 41.82  |  1.92  | 0.0490 
125  | 38.61  |  1.65  | 0.0659 
150  | 34.40  |  0.49  | 0.0720 
175  | 33.51  |  0.18  | 0.0780 
200  | 31.27  |  0.09  | 0.0841 
225  | 29.26  |  0.06  | 0.0899 
250  | 28.68  |  0.04  | 0.0957 
275  | 25.56  |  0.03  | 0.1015 
300  | 24.46  |  0.03  | 0.1095 
325  | 23.15  |  0.02  | 0.3093 
350  | 22.16  |  0.02  | 0.3373 
375  | 22.01  |  0.02  | 0.3513 
400  | 20.96  |  0.01  | 0.3573 
425  | 20.51  |  0.01  | 0.3597 
450  | 19.56  |  0.01  | 0.3621 
475  | 19.40  |  0.00  | 0.3644 
500  | 19.22  |  0.00  | 0.3710 
525  | 19.17  |  0.00  | 0.3733 
550  | 18.12  |  0.00  | 0.3758 
575  | 8.77  |  0.01  | 0.3782 
600  | 8.77  |  0.00  | 0.3814 


In [21]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [26]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

w8a


In [13]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]